In [1]:
import xarray as xr
import matplotlib.pyplot as plt
from cartopy import crs, feature
import numpy as np

In [2]:

#ds = xr.open_dataset('montotal.nc').drop_vars('time_bnds')
ds = xr.open_dataset('yeartotal.nc').drop_vars('time_bnds')
ds = ds.sel(time=slice('1975','2023'))


In [3]:
ds


<xarray.Dataset>
Dimensions:    (time: 49, longitude: 23, latitude: 37)
Coordinates:
  * time       (time) datetime64[ns] 1975-07-02 ... 2023-07-02
  * longitude  (longitude) float32 -3.41 -3.19 -2.97 -2.75 ... 0.99 1.21 1.43
  * latitude   (latitude) float32 4.51 4.73 4.95 5.17 ... 11.99 12.21 12.43
Data variables:
    eir        (time, latitude, longitude) float32 ...
Attributes: (12/77)
    CDI:                    Climate Data Interface version 1.9.9rc1 (https://...
    Conventions:            CF-1.6
    history:                Tue Aug 01 10:56:07 2023: cdo yearsum dta.nc year...
    origin:                 Adrian Tompkins, ICTP, Strada Costiera 11, Triest...
    url:                    www.ictp.it/~tompkins/vectri
    email:                  tompkins@ictp.it
    ...                     ...
    rtemperature_trend:     0.0
    rrainfall_factor:       1.0
    rmigration:             1e-05
    rhost_infect_init:      0.1
    frequency:              year
    CDO:                    Climate Data Operators version 1.9.9rc1 (https://...

In [4]:
Ghana_ds = ds.sel(longitude=slice(-3.4,1.43),latitude=slice(4.5,12.43)).mean(['longitude','latitude'])
#Ghana_ds = Ghana_ds.sel(time=slice('2000','2023'))



In [5]:
# import os
# import re

# times_x = ds.time.data
# tt_blank = np.ones([3, times_x.size]) * np.nan
# rr_blank = np.zeros(times_x.size)


# # Function to extract the year from the time string
# def extract_year(time_str):
#     return time_str[:4]

# for j, time in enumerate(ds.time.data[0:]):
#     for i, cl in enumerate(range(-4, 4, 12)):
#         plt.figure(figsize=(25, 15))
#         plt.subplots_adjust(hspace=0.5, wspace=0.2)
#         ax = plt.subplot(221, projection=crs.PlateCarree(central_longitude=cl))
#         ds['eir'].sel(time=time).plot(ax=ax, cmap='coolwarm', transform=crs.PlateCarree())
#         ax.add_feature(feature.COASTLINE)
#         ax.add_feature(feature.BORDERS)
#         ax.add_feature(feature.STATES, linewidth=0.2)
#         ax.set_xticks([-3, -2.0, -1.0, 0, 0.8], crs=crs.PlateCarree())
#         ax.set_yticks([11, 10, 9, 8, 7, 6, 5, 4], crs=crs.PlateCarree())
#         ax = plt.subplot(222)
#         regions = ['Ghana']
#         for r, reg in enumerate(regions):
#             if j == 0:
#                 mon = eval(reg + '_ds').eir.sel(time=time)
#             else:
#                 mon = eval(reg + '_ds').eir.sel(time=slice(None, time))

#             tt_blank[r, :mon.size] = mon.values
#             ax.plot(range(len(tt_blank[r])), tt_blank[r], label=reg)
#         ax.set_xticks(range(len(times_x)))
#         ax.set_xticklabels([extract_year(str(t)) for t in times_x], rotation=90)

#         # Save the figure with the year as filename
#         filename = extract_year(str(times_x[j])) + ".png"
#         plt.savefig(os.path.join("Animation", filename),facecolor='grey')

#         # Close the figure to release resources and avoid overwriting plots
#         plt.close()


In [93]:
import os
import re
from PIL import Image

times_x = ds.time.data
tt_blank = np.ones([3, times_x.size]) * np.nan
rr_blank = np.zeros(times_x.size)

# Create a directory to save the figures
if not os.path.exists("Animation"):
    os.makedirs("Animation")

# Function to extract the year from the time string
def extract_year(time_str):
    return time_str[:4]

# List to store image paths for GIF creation
image_paths = []

for j, time in enumerate(ds.time.data[0:]):
    for i, cl in enumerate(range(-4, 4, 12)):
        plt.figure(figsize=(25, 15))
        plt.subplots_adjust(hspace=0.5, wspace=0.2)
        ax = plt.subplot(221, projection=crs.PlateCarree(central_longitude=cl))
        ds['eir'].sel(time=time).plot(ax=ax, cmap='coolwarm', transform=crs.PlateCarree(),vmax=390)
        ax.add_feature(feature.COASTLINE)
        ax.add_feature(feature.BORDERS)
        ax.add_feature(feature.STATES, linewidth=0.2)
        ax.set_xticks([-3, -2.0, -1.0, 0, 0.8], crs=crs.PlateCarree())
        ax.set_yticks([11, 10, 9, 8, 7, 6, 5, 4], crs=crs.PlateCarree())
        ax = plt.subplot(222)
        regions = ['Ghana']
        for r, reg in enumerate(regions):
            if j == 0:
                mon = eval(reg + '_ds').eir.sel(time=time)
            else:
                mon = eval(reg + '_ds').eir.sel(time=slice(None, time))

            tt_blank[r, :mon.size] = mon.values
            ax.plot(range(len(tt_blank[r])), tt_blank[r], label=reg)
        ax.set_xticks(range(len(times_x)))
        ax.set_xticklabels([extract_year(str(t)) for t in times_x], rotation=90)
        ax.set_ylim(0, 300)
        plt.suptitle(f'{extract_year(str(times_x[j]))}', fontsize=30)
        # Save the figure with the year as filename
        filename = os.path.join("Animation", f"{extract_year(str(times_x[j]))}.png")
        plt.savefig(filename,facecolor='grey')
        image_paths.append(filename)

        # Close the figure to release resources and avoid overwriting plots
        plt.close()

# Create animated GIF from the images
images = [Image.open(path) for path in image_paths]
images[0].save("year_by_year_animation.gif", save_all=True, append_images=images[1:], optimize=False, duration=1000, loop=0)

# Remove the individual image files
for path in image_paths:
    os.remove(path)
